In [15]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib as plt
from tensorflow.keras import layers,models

In [16]:
dataset_path = 'Desktop/potato/dataset/PotatoPlants'
image_size = 256
batch_size = 32
channels = 3

input_shape = (batch_size,image_size,image_size,channels)

dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    batch_size=batch_size,
    image_size=(image_size, image_size)
)

    

Found 2152 files belonging to 3 classes.


In [17]:
class_names = dataset.class_names
class_names

['Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy']

In [18]:
len(dataset)

68

In [19]:
def get_train_test_val(train=0.8,test=0.1,val=0.1,shuffle=True,shuffle_size=10000):
    dataset_size = len(dataset)
    if shuffle:
        dataset.shuffle(shuffle_size, seed = 8)
        
    train_size = int(train*dataset_size)
    val_size = int(val*dataset_size)

    train_dataset = dataset.take(train_size)

    val_dataset = dataset.skip(train_size).take(val_size)
    test_dataset = dataset.skip(train_size).skip(val_size)
    
    return train_dataset,val_dataset,test_dataset


In [20]:
train,test,val = get_train_test_val()

In [21]:
len(train)

54

In [22]:
len(test)

6

In [23]:
len(val)

8

In [24]:
n_class = 3
model = tf.keras.Sequential([
    layers.Resizing(256,256),
    layers.Rescaling(1.0/255),
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
    layers.Conv2D(32,[3,3],activation='relu',input_shape=input_shape),
    layers.MaxPool2D(pool_size=(2,2)),
    layers.Conv2D(64,[3,3],activation='relu'),
    layers.MaxPool2D(pool_size=(2,2)),
    layers.Conv2D(64,[3,3],activation='relu'),
    layers.MaxPool2D(pool_size=(2,2)),
    layers.Conv2D(64,[3,3],activation='relu'),
    layers.MaxPool2D(pool_size=(2,2)),
    layers.Flatten(),
    layers.Dense(64,activation='relu'),
    layers.Dense(n_class,activation='softmax'),
    
])

In [25]:
model.build(input_shape=input_shape)

In [26]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ resizing_1 (Resizing)                │ (32, 256, 256, 3)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ rescaling_1 (Rescaling)              │ (32, 256, 256, 3)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ random_flip_1 (RandomFlip)           │ (32, 256, 256, 3)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ random_rotation_1 (RandomRotation)   │ (32, 256, 256, 3)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (32, 254, 254, 32)          │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (32, 127, 127, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (32, 125, 125, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (32, 62, 62, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_6 (Conv2D)                    │ (32, 60, 60, 64)            │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_6 (MaxPooling2D)       │ (32, 30, 30, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_7 (Conv2D)                    │ (32, 28, 28, 64)            │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_7 (MaxPooling2D)       │ (32, 14, 14, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (32, 12544)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (32, 64)                    │         802,880 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (32, 3)                     │             195 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 896,323 (3.42 MB)

 Trainable params: 896,323 (3.42 MB)

 Non-trainable params: 0 (0.00 B)

In [27]:
model.compile(
    optimizer = 'adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [28]:
history = model.fit(
    train,
    epochs=20,
    batch_size=batch_size,
    verbose=1,
    validation_data=val
)

Epoch 1/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 57s 1s/step - accuracy: 0.5973 - loss: 0.8075 - val_accuracy: 0.8491 - val_loss: 0.4597
Epoch 2/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 56s 1s/step - accuracy: 0.8442 - loss: 0.4153 - val_accuracy: 0.6379 - val_loss: 0.6438
Epoch 3/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 56s 1s/step - accuracy: 0.8381 - loss: 0.3754 - val_accuracy: 0.8836 - val_loss: 0.2826
Epoch 4/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 55s 1s/step - accuracy: 0.8497 - loss: 0.3323 - val_accuracy: 0.8966 - val_loss: 0.2957
Epoch 5/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 56s 1s/step - accuracy: 0.8908 - loss: 0.2659 - val_accuracy: 0.9310 - val_loss: 0.2220
Epoch 6/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 56s 1s/step - accuracy: 0.9158 - loss: 0.2028 - val_accuracy: 0.9181 - val_loss: 0.2488
Epoch 7/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 56s 1s/step - accuracy: 0.9093 - loss: 0.2161 - val_accuracy: 0.9181 - val_loss: 0.2162
Epoch 8/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 55s 1s/step - accuracy: 0.9375 - loss: 0.1698 - val_accuracy: 0.8362 - val_loss:

In [29]:
history

In [30]:
print(history.history.keys())

dict_keys(['accuracy', 'loss', 'val_accuracy', 'val_loss'])


In [31]:
history.history['accuracy'][-1]

0.9646990895271301

In [32]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Load and preprocess the image
img_path = 'Desktop/potato/dataset/PlantVillage/PlantVillage/Potato___Early_blight/02578b86-b234-4ac0-9bc3-691b5610e2bf___RS_Early.B 7562.JPG'
img = image.load_img(img_path, target_size=(256, 256))  # Adjust target_size to match your model's input size

# Convert the image to a numpy array
img_array = image.img_to_array(img)

# Normalize pixel values (if your model was trained with normalized images)
img_array /= 255.0  # Scale the pixel values to [0, 1]

# Add a batch dimension (the model expects input in batches)
img_array = np.expand_dims(img_array, axis=0)


In [35]:
# Make a prediction
prediction = model.predict(img_array)

# Interpret the result (depending on your model)
print("Prediction:", prediction)

# If it's a classification model
predicted_class = np.argmax(prediction, axis=1)
print("Predicted class:", predicted_class)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Prediction: [[8.6576022e-02 9.1337913e-01 4.4846343e-05]]
Predicted class: [1]


In [ ]:
model_version = 2
model.save(f""